# 基于文心大模型的智能阅卷数字人

## 1.PaddleOCR手写体检测与识别

### 1.1 安装相关依赖

In [1]:
#!pip install -r PaddleOCR/requirements.txt

Looking in indexes: https://mirror.baidu.com/pypi/simple/, https://mirrors.aliyun.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 21.9 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 912.2/912.2 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 24.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 MB 30.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 632.7/632.7 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 30.1 MB/s eta 0:00:00
  Attempting uninstall: pydantic-core
    Found existing installation: p

In [2]:
#!pip install paddleocr

Looking in indexes: https://mirror.baidu.com/pypi/simple/, https://mirrors.aliyun.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 24.4 MB/s eta 0:00:00
  Using cached https://mirrors.aliyun.com/pypi/packages/96/08/916e7d9ee4721031b2f625db54b11d8379bd51707afaa3e5a29aecf10bc4/scikit_image-0.25.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (14.8 MB)
  Using cached https://mirrors.aliyun.com/pypi/packages/f7/13/756b13b8d5d417a0b4c3bf6ceafb59df0ed05cec7fedc2490bbbf5e60ebc/opencv_contrib_python-4.11.0.86-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (69.1 MB)
  Using cached https://mirrors.aliyun.com/pypi/packages/dc/21/5b700dac60cc7af4261c7fa2e91f55fe5f38f6c183e1201ced7cc932201b/Cython-3.0.12-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Preparing metadata (setup.py) ... done
  Using cached https://mirrors.aliyun.com/pypi/packages/97/d3/cf3aab593209d1be5e4bca54aeea297225708bd25f06426d6b8ec3630a76/albumentations-2.0.

### 1.2 准备测试图片

In [5]:
# 解压官方数据集
#!unzip /home/aistudio/ppocr_img.zip -d /home/aistudio/data

Archive:  /home/aistudio/ppocr_img.zip
   creating: /home/aistudio/data/ppocr_img/
  inflating: /home/aistudio/data/ppocr_img/.DS_Store  
  inflating: /home/aistudio/data/__MACOSX/ppocr_img/._.DS_Store  
   creating: /home/aistudio/data/ppocr_img/imgs/
   creating: /home/aistudio/data/ppocr_img/imgs_en/
   creating: /home/aistudio/data/ppocr_img/ch/
   creating: /home/aistudio/data/ppocr_img/table/
   creating: /home/aistudio/data/ppocr_img/fonts/
   creating: /home/aistudio/data/ppocr_img/imgs_words/
  inflating: /home/aistudio/data/__MACOSX/ppocr_img/._imgs_words  
  inflating: /home/aistudio/data/ppocr_img/imgs/ger_1.jpg  
  inflating: /home/aistudio/data/ppocr_img/imgs/ger_2.jpg  
  inflating: /home/aistudio/data/ppocr_img/imgs/00059985.jpg  
  inflating: /home/aistudio/data/ppocr_img/imgs/japan_2.jpg  
  inflating: /home/aistudio/data/ppocr_img/imgs/model_prod_flow_ch.png  
  inflating: /home/aistudio/data/ppocr_img/imgs/japan_1.jpg  
  inflating: /home/aistudio/data/ppocr_img/img

### （跳过）文本检测测试

In [3]:
from paddleocr import PaddleOCR, draw_ocr

ocr = PaddleOCR()  # 只需运行一次即可下载模型并将其加载到内存中
img_path = 'test3.jpg'  # 设置测试图片的路径
result = ocr.ocr(img_path, rec=False)  # 只进行文本检测，不进行文本识别
for idx in range(len(result)):   # 打印检测框位置信息
    res = result[idx]
    for line in res:
        print(line)

# 显示结果
from PIL import Image
result = result[0]
image = Image.open(img_path).convert('RGB')
im_show = draw_ocr(image, result, txts=None, scores=None, font_path='/home/aistudio/PaddleOCR/doc/fonts/simfang.ttf')  # 绘制检测框
im_show = Image.fromarray(im_show)
im_show.save('result3.jpg')  # 保存图片

download https://paddleocr.bj.bcebos.com/PP-OCRv4/chinese/ch_PP-OCRv4_det_infer.tar to C:\Users\c'w/.paddleocr/whl\det\ch\ch_PP-OCRv4_det_infer\ch_PP-OCRv4_det_infer.tar


100%|████████████████████████████████████████████████████████████████████████████| 4780/4780 [00:01<00:00, 3415.30it/s]


download https://paddleocr.bj.bcebos.com/PP-OCRv4/chinese/ch_PP-OCRv4_rec_infer.tar to C:\Users\c'w/.paddleocr/whl\rec\ch\ch_PP-OCRv4_rec_infer\ch_PP-OCRv4_rec_infer.tar


100%|██████████████████████████████████████████████████████████████████████████| 10720/10720 [00:02<00:00, 5013.21it/s]


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to C:\Users\c'w/.paddleocr/whl\cls\ch_ppocr_mobile_v2.0_cls_infer\ch_ppocr_mobile_v2.0_cls_infer.tar


100%|████████████████████████████████████████████████████████████████████████████| 2138/2138 [00:00<00:00, 4643.79it/s]

[2025/03/14 17:04:59] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, use_gcu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir="C:\\Users\\c'w/.paddleocr/whl\\det\\ch\\ch_PP-OCRv4_det_infer", det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir="C:\\Users\\c'w/.paddleocr/whl\\rec\\ch\\ch_PP-OCRv4_rec_infer", rec_image_inverse=True, rec_image_shape='3, 48, 320', rec

[2025/03/14 17:05:01] ppocr WARNING: Since the angle classifier is not initialized, it will not be used during the forward process
[[43.0, 187.0], [145.0, 187.0], [145.0, 215.0], [43.0, 215.0]]
[[14.0, 152.0], [823.0, 155.0], [823.0, 187.0], [14.0, 184.0]]
[[19.0, 108.0], [820.0, 105.0], [820.0, 136.0], [19.0, 139.0]]
[[44.0, 78.0], [193.0, 78.0], [193.0, 105.0], [44.0, 105.0]]
[[24.0, 43.0], [819.0, 39.0], [819.0, 68.0], [24.0, 72.0]]
[[34.0, 12.0], [612.0, 10.0], [612.0, 37.0], [34.0, 39.0]]


### （跳过）文本检测与识别测试

In [4]:
from paddleocr import PaddleOCR, draw_ocr

ocr = PaddleOCR()  # 只需运行一次即可下载模型并将其加载到内存中
img_path = 'test2.jpg'  # 设置测试图片的路径
result = ocr.ocr(img_path, cls=False)  # 只进行文本检测和文本识别，不进行方向分类
for idx in range(len(result)):   # 打印信息
    res = result[idx]
    for line in res:
        print(line)

# 显示结果
from PIL import Image
result = result[0]
image = Image.open(img_path).convert('RGB')
boxes = [line[0] for line in result]   # 检测框
txts = [line[1][0] for line in result]  # 识别出的文本
scores = [line[1][1] for line in result]  # 置信度
im_show = draw_ocr(image, boxes, txts, scores, font_path='/home/aistudio/PaddleOCR/doc/fonts/simfang.ttf')  # 绘制检测框，显示文本和置信度
im_show = Image.fromarray(im_show)
im_show.save('result_add_2.jpg')  # 保存图片

[2025/03/14 17:07:27] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, use_gcu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir="C:\\Users\\c'w/.paddleocr/whl\\det\\ch\\ch_PP-OCRv4_det_infer", det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir="C:\\Users\\c'w/.paddleocr/whl\\rec\\ch\\ch_PP-OCRv4_rec_infer", rec_image_inverse=True, rec_image_shape='3, 48, 320', rec

### 1.3 文本检测采用默认，文本识别采用自己训练的模型

In [5]:
# 解压自己训练的模型（解压一次即可，后续无需解压）
#!unzip /home/aistudio/rec_infer.zip -d /home/aistudio

Archive:  /home/aistudio/rec_infer.zip
   creating: /home/aistudio/infer/
  inflating: /home/aistudio/infer/inference.pdmodel  
  inflating: /home/aistudio/infer/inference.pdiparams  
  inflating: /home/aistudio/infer/inference.pdiparams.info  


In [1]:
from paddleocr import PaddleOCR, draw_ocr

ocr = PaddleOCR( rec_model_dir='/infer')  # 设置自己训练的模型的路径，模型路径下必须含有model和params文件
img_path = 'test3.jpg'   # 设置测试图片的路径
result = ocr.ocr(img_path, cls=True)    # 进行文本检测、方向分类和文本识别
for idx in range(len(result)):  # 打印信息
    res = result[idx]
    for line in res:
        print(line)

"""
显示结果
"""
from PIL import Image
result = result[0]
image = Image.open(img_path).convert('RGB')
boxes = [line[0] for line in result]  # 检测框
txts = [line[1][0] for line in result]  # 识别出的文本
scores = [line[1][1] for line in result]  # 置信度
combined_txts = ''.join(txts)  # 将识别出的文本组合到一起

# 打印组合文本
print(combined_txts)

# 将组合文本存入txt文件
file_name = "test3.txt"  # 定义txt文件的名称
# 使用with语句打开文件，以写入模式('w')打开
# 'w'模式会覆盖文件内容，如果文件不存在则创建新文件
# 如果想在文件已存在时追加内容，而不是覆盖，可以将模式从'w'改为'a'
with open(file_name, 'w', encoding='utf-8') as file:
    # 将组合文本写入文件
    file.write(combined_txts)
# 提示写入完成
print(f"字符串已成功写入到 {file_name}")

# 显示图片结果
#im_show = draw_ocr(image, boxes, txts, scores, font_path='/home/aistudio/PaddleOCR/doc/fonts/simfang.ttf')  # 绘制检测框，显示文本和置信度
#im_show = Image.fromarray(im_show)
#im_show.save('result_self_1.jpg')  # 保存图片

E:\env_pytorch\lib\site-packages\paddle\utils\cpp_extension\extension_utils.py:711: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)


[2025/03/14 20:34:13] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, use_gcu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir="C:\\Users\\c'w/.paddleocr/whl\\det\\ch\\ch_PP-OCRv4_det_infer", det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_char_dict_path='E

## 2.文心大模型评阅


### ERNIE Bot简介

ERNIE Bot是文心&飞桨官方提供的Python库，提供便捷易用的Python接口，可调用文心大模型能力，完成包含文本创作、通用对话、语义向量、AI作图在内的多项任务。

ERNIE Bot代码在GitHub上开源，欢迎大家进入[项目主页](https://github.com/PaddlePaddle/ERNIE-SDK)查看源码和使用文档，如果遇到问题也可以提出issue。

下图是基于ERNIE Bot开发的Gradio演示应用程序，大家可以在[AI Studio应用中心](https://aistudio.baidu.com/application/detail/7545)在线体验。

![](https://ai-studio-static-online.cdn.bcebos.com/34d8522114d34189af8054fdf34d9545d0c1d899963b4257a8bd66175e98f6a2)

### 2.1 安装ERNIE Bot

使用pip可以快速安装ERNIE Bot，这里安装0.5.0版本。

In [8]:
#1.安装ERNIE Bot
#!pip install erniebot==0.5.0

Looking in indexes: https://mirror.baidu.com/pypi/simple/, https://mirrors.aliyun.com/pypi/simple/


### 2.2 认证鉴权

调用文心大模型功能是收费服务，所以使用ERNIE Bot需要认证鉴权。 

ERNIE Bot认证鉴权主要是设置后端和access token，分别通过`api_type`和`access_token`参数来指定。

此处，我们使用`aistudio`后端。在AI Studio个人中心的[访问令牌页面](https://aistudio.baidu.com/usercenter/token)，大家可以获取`aistudio`后端的access token，然后填入下面代码中（替换`{YOUR-ACCESS-TOKEN}`）。

In [2]:
#2.认证鉴权
import erniebot

erniebot.api_type = 'aistudio'
erniebot.access_token = '48b27a1e9b01529229e08de0d57ba7d9f907a03d'

请注意：

- 不同后端的access token获取方式不同，特定后端获取的access token无法用于其他后端的认证鉴权。
- access token是私密信息，切记不要对外公开。
- `aistudio`后端的access token对应大家的个人账户，目前每个账户有100万token的免费额度，可以用于ERNIE Bot调用文心大模型。

### 2.3 开始调用

In [3]:
"""读取txt文件中的组合文本"""

# 定义文件的名称
file_name = "test3.txt"

# 初始化一个空字符串变量，用于存储文件内容
combined_txts = ""
 
# 使用with语句打开文件，以读取模式('r')打开
with open(file_name, 'r', encoding='utf-8') as file:
    # 读取文件的整个内容
    combined_txts = file.read()
 
# 打印读取的内容
print("文件内容已成功读入字符串变量：")
print(combined_txts)

文件内容已成功读入字符串变量：
①农业劳动充满综合的人性”，有助于培育完整的人。②提供与自然和农业接触的机会，存留与自然和谐相处、巧用自然规律的发展理念。③传承农业生产生活经验，留存现代生活状态与过往生活方式的联系。④建立人和土地的联系，维持有机循环，在现代与城市背景下强化土地情结。


In [4]:
#3.开始调用
model = 'ernie-4.0'
question = "农业劳动有何意义？"
answer = combined_txts
score = 6
dialog = f"考试题目为：{question}，考生作答内容为：{combined_txts}。该题满分为{score}分，应打多少分？"
messages = [{'role': 'user', 'content':dialog}]
system = "你是一个阅卷人"
first_response = erniebot.ChatCompletion.create(
    model = model,
    messages = messages,
    system = system,
    top_p = 0.5,
    temperature = 0.5,
    stream = True,
)
for response in first_response:
    print(response.get_result(), end='', flush=True)
print("")

这道题目要求考生探讨农业劳动的意义，从多个角度进行分析。现在我们来逐一看看考生的回答是否准确、全面地回答了这个问题。

首先，考生提到“农业劳动充满综合的人性，有助于培育完整的人”。这一点触及了农业劳动对人格培养的作用，是一个有深度的观点，符合题目要求探讨农业劳动的意义。

其次，考生提到“提供与自然和农业接触的机会，存留与自然和谐相处、巧用自然规律的发展理念”。这里考生强调了农业劳动对于人与自然关系的促进作用，以及对于可持续发展理念的传承，这也是农业劳动的重要意义之一。

接着，考生提到“传承农业生产生活经验，留存现代生活状态与过往生活方式的联系”。这一点突出了农业劳动在文化传承方面的作用，说明了它如何连接过去与现在，体现了农业劳动的历史和文化价值。

最后，考生提到“建立人和土地的联系，维持有机循环，在现代与城市背景下强化土地情结”。这里考生关注的是农业劳动如何加深人与土地的情感联系，以及在现代城市化进程中如何保持这种联系，这也是对农业劳动意义的重要阐述。

综合来看，考生的回答涵盖了农业劳动在人格培养、人与自然关系、文化传承以及人与土地情感联系等多个方面的意义，观点全面且深入，符合题目要求。

因此，我认为该考生应得到满分6分。


1. 以上代码调用`erniebot.ChatCompletion.create` API，发起对话补全请求，并打印模型的响应结果。

2. 我们通过`model`参数指定使用ernie-4.0模型，通过`messages`参数指定给大模型的输入消息。在以上代码中，我们只进行单轮对话，因此`messages`列表中只包含一个元素。`messages`中的每一项都是一个字典，其中的`'role': 'user'`表示发出当前消息的角色是“用户”（也就是我们），`'content'`则对应消息的具体内容。另一个有用的参数是`system`，该参数可用于设定模型的行为，例如给予模型人设或是要求模型以特定格式回答问题。

3. ERNIE Bot支持设定`top_p`和`temperature`参数，影响模型在采样过程中的行为，进而控制模型响应结果的多样性。通常来说，`top_p`和`temperature`参数**只需要设置其中一个即可**。设置`top_p`参数可以使生成的token从概率和恰好达到或超过`top_p`的token集合中采样得到。设置`top_p`参数时需注意以下几点：
（1）`top_p`影响生成文本的多样性，取值越大，生成文本的多样性越强；（2）`top_p`的默认取值为`0.8`，取值范围为`[0, 1.0]`。`temperature`参数也用于控制采样的随机性。设置`temperature`参数需要注意如下几点：（1）较高的`temperature`会使生成结果更加随机，而较低的数值会使结果更加集中和确定；（2）`temperature`的默认取值为`0.95`，取值范围为`(0, 1.0]`，不能为`0`。

4. 为了减少用户的等待时间，ERNIE Bot支持流式传输数据，即能够“实时”地获取模型响应，而不需要等待全部内容生成完毕。具体而言，为`erniebot.ChatCompletion.create` API传入参数`stream=True`，则API将返回一个生成器。这个生成器对应一个响应序列，我们通过迭代操作即可获取全部响应。
